*Created by Petteri Nevavuori <<petteri.nevavuori@gmail.com>>*

---

**CHIO & FREEMAN: MACHINE LEARNING & SECURITY (2018)** <br>
*<small>Otsikot kirjan mukaan, muutoin suomeksi.</small>*

<h1>Sisällysluettelo<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Classifying-and-Clustering" data-toc-modified-id="Classifying-and-Clustering-1">Classifying and Clustering</a></span><ul class="toc-item"><li><span><a href="#Machine-Learning:-Problems-and-Approaches" data-toc-modified-id="Machine-Learning:-Problems-and-Approaches-1.1">Machine Learning: Problems and Approaches</a></span></li><li><span><a href="#Machine-Learning-in-Practice:-A-Worked-Example" data-toc-modified-id="Machine-Learning-in-Practice:-A-Worked-Example-1.2">Machine Learning in Practice: A Worked Example</a></span></li></ul></li></ul></div>

# Classifying and Clustering

Tässä luvussa käydään läpi tietoturvasovellusten käyttökelpoisimpia koneoppimismenetelmiä, niiden perusteista aina käyttökelpoisen koneoppimistyökalupakin rakentamiseen. Luvussa pysyttäydytään menetelmien toiminnan ymmärtämisen tasossa menemättä nyansseihin.

## Machine Learning: Problems and Approaches

Koneoppimismenetelmät ovat pohjimmiltaan algoritmeja ja historiallista dataa hyödyntäviä prosesseja tulevaisuuden tapahtumien luokitteluun (*classification*) tai arvojen ennustamiseen (*regression*). Luokittelu voi olla *binääristä* joko-tai-luokittelua tai sitten luokittelua yhdeksi tai useaksi monesta mahdollisesta luokasta (*multiclass classification*).

Tietoturvaan liittyvät ongelmat liittyvät suuressa määrin jatkuvaan havainnointiin, jossa kukin uusi tilanne on kyettävä luokittelemaan joko turvalliseksi tai uhaksi. Näitä tilanteita ovat esimerkiksi jokainen uusi tiedosto, sähköposti, kirjautumisyritys ja palvelimelle tullut pyyntö. Ihmisvoimin käsitellen tehtävä osoittautuu piankin mahdottomaksi, mutta käyttämällä dataa eli vaikkapa kirjautumistietojen, tiedostojen, yhteydenottopyyntöjen jne. lokitiedostoja (*logs*) yhdessä datasta piirteitä oppivien mallien kanssa tehtävä helpottuu ja voidaan myös automatisoida.

Arvojen ennustaminen eli regressio tulee kyseeseen, kun halutaan saada luotettava arvio vaikkapa johonkin työntekijään kohdistuneista kalasteluyrityksistä (*phishing*) kuukautta kohden. Tällöin logien sijasta käytetään muita muuttujia, kuten henkilön asema työpaikalla, käyttöoikeudet, viran pituus yms. Mikäli taas arvoja ennustetaan datasta, jossa on mukana myös ajan ulottuvuus eli peräkkäisiä havaintoja joiden oletetaan vaikuttavan ennustettavaan tulokseen, ollaan tekemisissä aikasarja-analyysin (*time series analysis*) kanssa. Tietoturvakonteksissa aikasarja-analyysiä voidaan käyttää ennustamaan normaalia kirjautumisten määrää vaikkapa jostain toimipisteestä, mitä sitten voidaan verrata ja tarvittaessa huomata mahdollinen kirjautumisten määrän poikkeava kasvu. Esimerkeistä viimeisin on hyvä esimerkki samalla myös poikkeamien havaitsemisesta (*anomaly detection*).

Poikkeamia voidaan havaita myös ryhmittelymenetelmillä (*clustering*). Tällöin menetelmälle syötetty data ryhmitellään eli *klusteroidaan* havaintojen samankaltaisuuden perusteella suuremmiksi pääryhmiksi. Ryhmittelymenetelmälle voidaan opettaa esimerkiksi turvallisia tapahtumia. Sen kohdatessa uhaksi tulkittavan havainnon jää kyseinen havainto opittujen ryhmien ulkopuolelle. Ryhmittelyllä voidaan saada ymmärrystä myös vaikkapa sivustolle lähetetyistä pyynnöistä - mitkä pyynnöistä ovat todennäköisesti bottiverkon (*botnet*) lähettämiä pyyntöjä, mitkä taas virallisia.

Koneoppiminen jaetaan perinteisesti myös ohjattuihin (*supervised*) ja ohjaamattomiin (*unsupervised*) menetelmiin. Luokittelu- ja regressiomenetelmät ovat perinteisesti ohjattuja, jolloin niitä koulutettaessa menetelmille syötetään havaintotiedon lisäksi myös oikea vastaus - luokka tai lukuarvo. Käytännön esimerkki ohjatuista menetelmistä on edellisen luvun roskapostien tunnistaminen, jossa luokka oli ennalta jo tiedossa. Ryhmittelymenetelmät pohjaavaat lähes aina dataan itseensä etsien ja ryhmitellen dataa sen sisäisten ominaisuuksien perusteella. Tämä on ohjaamatonta oppimista, eli menetelmän annetaan itse määrittää datalle ominaiset ryhmät ja niiden ominaisuudet.

## Machine Learning in Practice: A Worked Example

Tässä esimerkissä käytetään ostotapahtumista (*transaction*) muodostettua datasettiä, joka löytyy [kirjan GitHubista](https://github.com/oreilly-mlsec/book-resources/blob/master/chapter2/datasets/payment_fraud.csv). Datasetissä on viisi piirrettä talletettuna kustakin tapahtumasta sekä tieto siitä, oliko ostotapahtuma petos (*fraud*) vai ei. Tieto siitä, onko jokin ostotapahtuma ollut petos vai ei, on saatu keräämällä tietoa tapahtumiin liittyvistä mitätöintipyynnöistä - mikäli käyttäjä on epäillyt maksuvälineensä väärinkäyttöä, hän on pyytänyt tehdyn ostoksen mitätöinti ja rahojen palautusta. Pyynnöt eivät kuitenkaan ole reaaliaikaisesti käytettävissä, sillä niitä tehdään jopa kuukausia tapahtuman jälkeen. Tällöin myös ostetut tavarat on ehditty jo lähettää, eikä niitä juuri saada enää takaisin. Siksi onkin merkittävää kyetä ennustamaan, mikäli jokin ostotapahtuma ilmentää petollisia piirteitä ja estää mahdollinen liiketoimintahäviö jo ennen tavaran lähetystä.

Ensin ladataan tiedot käyttöön ja tarkastellaan muutamaa ensimmäistä riviä.

In [7]:
import os
import pandas as pd

ROOT_DIR = os.path.dirname(os.getcwd())
DATASET_PATH = os.path.join(ROOT_DIR,'datasets','payment_fraud.csv')

dataset = pd.read_csv(DATASET_PATH)
dataset.head()

,accountAgeDays,numItems,localTime,paymentMethod,paymentMethodAgeDays,label
0,29,1,4.745402,paypal,28.204861,0
1,725,1,4.742303,storecredit,0.000000,0
2,845,1,4.921318,creditcard,0.000000,0
3,503,1,4.886641,creditcard,0.000000,0
4,2000,1,5.040929,creditcard,0.000000,0


Datasetin muuttujat ovat yhtä lukuunottamatta sellaisenaan käytettävässä muodossa. Maksutapaa ilmaiseva `paymentMethod` sisältää kuitenkin kategorisia arvoja, joiden kanssa on toimittava erityistä huomiota kiinnittäen. Koska koneoppimismenetelmät toimivat luvuilla, on tekstimuotoiset tiedot ensinnäkin muutettava luvuiksi. Pelkkä `paypal => 1, storecredit => 2, ...` muunnos ei kuitenkaan ole kelvollinen, sillä muuttujilla ei ole lineaarista suhdetta toisiinsa - Paypal ei ole maksuvälineenä suurempi, parempi tai huonompi, kuin käteinen.

Kategorisia muuttujia sisältävä sarake tavataan usein hajoittaa useaksi sarakkeeksi siten, että jokainen hajoitettu sarake vastaa yhtä alkuperäisen sarakkeen uniikkia arvoa. Toisin sanoen sarake

| Sankarit |
| -------- |
| Nalle Puh |
| Hämis |
| Mustanaamio |

hajoitettaisiin useaksi sarakkeeksi seuraavasti:

| Sankarit: Nalle Puh | Sankarit: Hämis | Sankarit: Mustanaamio |
| -------- | -------- | -------- | 
| 1 | 0 | 0 |
| 0 | 1 | 0 |
| 0 | 0 | 1 |

Tämä tunnetaan valemuuttujien (*dummy variables*) muodostamisena sekä [*one-hottauksena*](https://en.wikipedia.org/wiki/One-hot). Valemuuttujien muodostamisen lisäksi, vaikka kirjassa tätä ei toistaiseksi käsitellä, on myös tavallista varmistua datasetin muuttujien *yksiselitteisyydestä*. Esimerkkimme taulukossa meillä on kolme arvoa ja loogisesti seuraten sen hajoitetussa versiossa kolme saraketta. Hajoitettu taulukko sisältää kuitenkin enemmän informaatiota, kuin mitä tarvitaan sankarin määrittämiseen yksiselitteisesti. Jos sankari on Nalle Puh tai Hämis, arvo on yksiselitteinen. Jos sankari ei ole kumpikään näistä, se on edelleen yksiselitteinen, sillä ainoa jäljelle jäävä vaihtoehto on Mustanaamio. Tieto Mustanaamiosta on siis ylimääräinen ja se itseasiassa aiheuttaa mallin [piirteiden selityskykyä alentavan ongelman](https://en.wikipedia.org/wiki/Multicollinearity), vaikkakaan suorituskyky tarkkuuden näkökulmasta ei kärsisikään.

Siispä seuraavaksi korvataan `paymentMethod`-sarake sen arvoista muodostetuilla valesarakkeilla jättäen yksi sarakkeista pois.

In [8]:
print(f'Unique paymentMethod categories: {list(dataset.paymentMethod.unique())}')
dataset = pd.get_dummies(dataset, prefix='paymentMethod', columns=['paymentMethod'], drop_first=True)
dataset.head()

Unique paymentMethod categories: ['paypal', 'storecredit', 'creditcard']


,accountAgeDays,numItems,localTime,paymentMethodAgeDays,label,paymentMethod_paypal,paymentMethod_storecredit
0,29,1,4.745402,28.204861,0,1,0
1,725,1,4.742303,0.000000,0,0,1
2,845,1,4.921318,0.000000,0,0,0
3,503,1,4.886641,0.000000,0,0,0
4,2000,1,5.040929,0.000000,0,0,0


Datan esikäsittely on nyt valmis. Seuraavaksi jatketaan jo opittuun malliin jakaen datasetti koulutus ja testidatoihin. Koulutuksen kohteena (*target*) `y` käytetään datasetin `label` sarakkeen tietoja ja syötedatoina (*input*) `x` kaikkia muita sarakkeita.

In [9]:
from sklearn.model_selection import train_test_split

x, y = dataset.drop('label', axis=1), dataset.label
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1/3)

Datasettien jakamisen jälkeen valitaan luokitteluun sopiva logistinen regressiomalli (*logistic regression*) lineaaristen mallien perheestä ja koulutetaan eli sovitetaan (*fit*) se dataan. Sen jälkeen muodostetaan ennuste testisetin syötedatoista ja verrataan ennusteita todellisiin tietoihin sekaannusmatriisilla. Sarakkeen `label` arvot on hyvä vielä kerrata: nolla tarkoittaa oikeaa ostotapahtumaa, yksi taas petosta.

In [13]:
from sklearn.linear_model import LogisticRegression
from utils import train_utils

model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
train_utils.visualize_results_acc_cm(y_test, y_pred)

Tulokset määrissä:


,true_0,true_1
model_0,12900,0
model_1,0,174


Tulokset prosenteissa:


,true_0,true_1
model_0,98.669114,0.000000
model_1,0.000000,1.330886


Mallin tarkkuus (ON+OP): 100.000%


Kouluttamamme malli onnistui luokittelemaan ostotapahtumat niiden tekohetkellä sille syötetyn datasetin pohjalta täydellisesti. Huomionarvoista on kuitenkin, että tämä ei tarkoita sitä, että malli olisi täydellinen todellisessa käytössä - sitä pitäis kouluttaa uudelleen ja uudelleen aina uusien havaintojen myötä.